## LangChain


In [ ]:
pip install --upgrade openai


In [ ]:
# pip show openai

In [ ]:
pip install langchain

In [ ]:
pip install "langserve[all]"

In [ ]:
import os
import sys
import openai

In [ ]:
openai.api_key = 'sk-xxxxxxx' # enter your key
OPENAI_API_KEY = openai.api_key
#os.environ['OPENAI_API_KEY'] = openai.api_key


## LLM / Chat Model
There are two types of language models:

LLM: underlying model takes a string as input and returns a string

ChatModel: underlying model takes a list of messages as input and returns a message

LLM.invoke: Takes in a string, returns a string.
ChatModel.invoke: Takes in a list of BaseMessage, returns a BaseMessage

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI()

from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

LLM and ChatModel objects are effectively configuration objects. You can initialize them with parameters like temperature and others, and pass them around.

In [ ]:
from langchain.schema import HumanMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]

AIMessage(content='ColorfulStride')

In [ ]:
llm.invoke(text)
# >> Feetful of Fun

'\n\nBright Socks Co.'

In [ ]:
chat_model.invoke(messages)
# >> AIMessage(content="Socks O'Color")

AIMessage(content='SockSplash')

## Prompt templates
Most LLM applications do not pass user input directly into an LLM. Usually they will add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand.

PromptTemplates help with exactly this! They bundle up all the logic for going from user input into a fully formatted prompt. This can start off very simple - for example, a prompt to produce the above string would just be:


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")


'What is a good name for a company that makes colorful socks?'

PromptTemplates can also be used to produce a list of messages.

In this case, the prompt not only contains information about the content, but also each message (its role, its position in the list, etc.). Here, what happens most often is a ChatPromptTemplate is a list of ChatMessageTemplates. Each ChatMessageTemplate contains instructions for how to format that ChatMessage - its role, and then also its content. Let's take a look at this below:

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

## Output parsers

OutputParsers convert the raw output of a language model into a format that can be used downstream. There are few main types of OutputParsers, including:

-> Convert text from LLM into structured information (e.g. JSON)

-> Convert a ChatMessage into just a string

-> Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

Below we will write our own output parser - one that converts a comma separated list into a list.

In [ ]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

## Composing with LCEL
We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser. This is a convenient way to bundle up a modular piece of logic. Let's see it in action!

In [ ]:
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'orange']

In [ ]:
# from langchain.document_loaders import TextLoader
# from langchain.indexes import VectorstoreIndexCreator

In [ ]:
pip install chromadb

In [ ]:
pip install tiktoken

In [ ]:
loader = TextLoader('Data.txt')

In [ ]:
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
query = "what is Suella Braverman claim"
print(index.query(query, llm=ChatOpenAI()))